In [2]:
import pandas 
import json

def parse_five_prime_utr_variant_consequence(conseq_str):
    """
    Parses the consequence str into a keyed dictionary as per
    https://github.com/ImperialCardioGenetics/UTRannotator#the-detailed-annotation-for-each-consequence

    """
    return {
        annotation.split(':')[0]: annotation.split(':')[1]
        for annotation in conseq_str.split(',')
    }
orfs = pandas.read_csv('../../../data/pipeline/ORFS_Features_1.0.tsv', sep="\t")
orfs['ensembl_transcript_id'] = orfs['ensembl_transcript_id'].apply(
        lambda x: x.split('.')[0]
    )
parsed_variants = pandas.read_csv('../../../data/pipeline/toy_dataset.txt', sep="\t")
parsed_variants['variant_id'] = parsed_variants['#Uploaded_variation']

sequences = pandas.read_csv('../../../data/pipeline/MANE_transcripts_v1.0.tsv', sep="\t")
sequences['ensembl_transcript_id'] = sequences['transcript_id'].apply(lambda x: x.split('.')[0])


/var/folders/zk/dmbrd2310jn8kdb96r649lr80000gn/T/ipykernel_875/2561380274.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [3]:
sequences

,transcript_id,five_prime_utr_length,three_prime_utr_length,num_five_prime_utr_exons,start_site_pos,cds_start,cds_end,cds_length,ensembl_transcript_id,annotations,...,gene,gene_biotype,transcript_biotype,gene_symbol,description,build,chr,start,end,strand
0,ENST00000000233.10,88,401,1,89,2,544,543,ENST00000000233,>ENST00000000233.10 cdna chromosome:GRCh38:7:1...,...,ENSG00000004059.11,protein_coding,protein_coding,ARF5,ADP ribosylation factor 5 [Source:HGNC Symbol;...,GRCh38,7,127588411,127591700,1
1,ENST00000000412.8,159,1457,2,160,3,836,834,ENST00000000412,>ENST00000000412.8 cdna chromosome:GRCh38:12:8...,...,ENSG00000003056.8,protein_coding,protein_coding,M6PR,"mannose-6-phosphate receptor, cation dependent...",GRCh38,12,8940361,8949645,-1
2,ENST00000000442.11,225,777,2,226,3,1274,1272,ENST00000000442,>ENST00000000442.11 cdna chromosome:GRCh38:11:...,...,ENSG00000173153.17,protein_coding,protein_coding,ESRRA,estrogen related receptor alpha [Source:HGNC S...,GRCh38,11,64305524,64316743,1
3,ENST00000001008.6,170,2165,1,171,2,1381,1380,ENST00000001008,>ENST00000001008.6 cdna chromosome:GRCh38:12:2...,...,ENSG00000004478.8,protein_coding,protein_coding,FKBP4,FKBP prolyl isomerase 4 [Source:HGNC Symbol;Ac...,GRCh38,12,2794970,2805423,1
4,ENST00000001146.7,28,2989,1,29,2,1540,1539,ENST00000001146,>ENST00000001146.7 cdna chromosome:GRCh38:2:72...,...,ENSG00000003137.9,protein_coding,protein_coding,CYP26B1,cytochrome P450 family 26 subfamily B member 1...,GRCh38,2,72129238,72147862,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19115,ENST00000699088.1,22,239,1,23,2,922,921,ENST00000699088,>ENST00000699088.1 cdna chromosome:GRCh38:7:10...,...,ENSG00000213413.3,protein_coding,protein_coding,PVRIG,PVR related immunoglobulin domain containing [...,GRCh38,7,100219949,100221490,1
19116,ENST00000699294.1,487,2911,2,488,3,1676,1674,ENST00000699294,>ENST00000699294.1 cdna chromosome:GRCh38:12:7...,...,ENSG00000180881.21,protein_coding,protein_coding,CAPS2,calcyphosine 2 [Source:HGNC Symbol;Acc:HGNC:16...,GRCh38,12,75275979,75330324,-1
19117,ENST00000699333.1,181,5137,1,182,2,1270,1269,ENST00000699333,>ENST00000699333.1 cdna chromosome:GRCh38:19:5...,...,ENSG00000251369.10,protein_coding,protein_coding,ZNF550,zinc finger protein 550 [Source:HGNC Symbol;Ac...,GRCh38,19,57541838,57559863,-1
19118,ENST00000699524.1,170,1490,2,171,3,1325,1323,ENST00000699524,>ENST00000699524.1 cdna chromosome:GRCh38:1:84...,...,ENSG00000122432.19,protein_coding,protein_coding,SPATA1,spermatogenesis associated 1 [Source:HGNC Symb...,GRCh38,1,84506386,84567379,1


In [4]:

snps = parsed_variants[parsed_variants['Allele'].isin(['A', 'C', 'G', 'T']) & 
                       ~parsed_variants['cDNA_position'].str.contains(r'\d+-\d+') &
                       parsed_variants['Feature'].str.contains('ENST00000358273')
                       ]
snps['annotation'] = snps['five_prime_UTR_variant_annotation'].apply(parse_five_prime_utr_variant_consequence)


/var/folders/zk/dmbrd2310jn8kdb96r649lr80000gn/T/ipykernel_875/808106407.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snps['annotation'] = snps['five_prime_UTR_variant_annotation'].apply(parse_five_prime_utr_variant_consequence)


In [5]:
snps[
    (snps['Allele'] == 'G') &
    (snps['five_prime_UTR_variant_consequence'] == 'uSTOP_lost')
]

,#Uploaded_variation,Location,Allele,Gene,Feature,Feature_type,Consequence,cDNA_position,CDS_position,Protein_position,...,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,existing_InFrame_oORFs,existing_OutOfFrame_oORFs,existing_uORFs,five_prime_UTR_variant_annotation,five_prime_UTR_variant_consequence,variant_id,annotation
2693,17-31095036-T-G,17:31095036,G,ENSG00000196712,ENST00000358273,Transcript,5_prime_UTR_variant,60,-,-,...,YES,NM_001042492.3,-,0,0,2,"uSTOP_lost_AltStop:False,uSTOP_lost_AltStopDis...",uSTOP_lost,17-31095036-T-G,"{'uSTOP_lost_AltStop': 'False', 'uSTOP_lost_Al..."
2694,17-31095036-T-G,17:31095036,G,ENSG00000196712,ENST00000358273,Transcript,5_prime_UTR_variant,60,-,-,...,YES,NM_001042492.3,-,0,0,2,"uSTOP_lost_AltStop:False,uSTOP_lost_AltStopDis...",uSTOP_lost,17-31095036-T-G,"{'uSTOP_lost_AltStop': 'False', 'uSTOP_lost_Al..."
2703,17-31095037-A-G,17:31095037,G,ENSG00000196712,ENST00000358273,Transcript,5_prime_UTR_variant,61,-,-,...,YES,NM_001042492.3,-,0,0,2,"uSTOP_lost_AltStop:False,uSTOP_lost_AltStopDis...",uSTOP_lost,17-31095037-A-G,"{'uSTOP_lost_AltStop': 'False', 'uSTOP_lost_Al..."
2704,17-31095037-A-G,17:31095037,G,ENSG00000196712,ENST00000358273,Transcript,5_prime_UTR_variant,61,-,-,...,YES,NM_001042492.3,-,0,0,2,"uSTOP_lost_AltStop:False,uSTOP_lost_AltStopDis...",uSTOP_lost,17-31095037-A-G,"{'uSTOP_lost_AltStop': 'False', 'uSTOP_lost_Al..."


In [73]:
snps[
    (snps['five_prime_UTR_variant_consequence'] == 'uSTOP_lost')
]['five_prime_UTR_variant_annotation'].values[15]

'uSTOP_lost_AltStop:False,uSTOP_lost_AltStopDistanceToCDS:NA,uSTOP_lost_Evidence:NA,uSTOP_lost_FrameWithCDS:outOfFrame,uSTOP_lost_KozakContext:GTGATGG,uSTOP_lost_KozakStrength:Strong'

In [74]:
def get_sequence_and_start(transcript_id):
    return sequences[sequences['ensembl_transcript_id'] == 'ENST00000358273']['seq'].values[0], sequences[sequences['ensembl_transcript_id'] == 'ENST00000358273']['start_site_pos'].values[0]

seq, start_site = get_sequence_and_start('ENST00000358273')
conti_seq = seq[62:]
subs = [conti_seq[i:i+3] for i in range(0, len(conti_seq), 3)]
subs.index('tga')

126

In [75]:
snps[
    (snps['Allele'] == 'T') &
    (snps['five_prime_UTR_variant_consequence'] == 'uAUG_gained')
]['cDNA_position'].values[0]

'54'

In [76]:
def find_impacted_orf(variant_pos, orfs):
    pass


def mutate_sequence(cdna_pos, ref, alt, type, seq):
    if type == 'SNV':
        return seq[:cdna_pos-1] + alt + seq[cdna_pos:]
    elif type == 'Deletion':
        return seq[:cdna_pos-1] + seq[cdna_pos+len(ref)-1:]
    elif type == 'Insertion':
        return seq[:cdna_pos-1] + alt + seq[cdna_pos-1:]
    else:
        raise ValueError('Unknown type')

def get_sequence_and_start(transcript_id):
    return sequences[sequences['ensembl_transcript_id'] == 'ENST00000358273']['seq'].values[0], sequences[sequences['ensembl_transcript_id'] == 'ENST00000358273']['start_site_pos'].values[0]

# uAUG_gained (Deletions, Insertions)
# uAUG_lost (SNVs, Deletions, Insertions)
# uStop_gained (SNVs, Deletions, Insertions)
# uStop_lost (SNVs, Deletions, Insertions)
# uFrameshift (SNVs, Deletions, Insertions)

def draw_track(consequence, annotation, seq, cdna_pos, ref, alt, start_site, variant_type):
    if consequence == 'uAUG_gained':
        # Find impacted ORF
        new_atg_pos = int(annotation['uAUG_gained_CapDistanceToStart'])
        stop_site =  int(annotation['uAUG_gained_CapDistanceToStart']) + int(annotation['uAUG_gained_DistanceToStop'])-1
        mutated_seq = mutate_sequence(cdna_pos, ref, alt, variant_type, seq)
        new_context = mutated_seq[new_atg_pos-6:new_atg_pos+5]
        return {
            'annotation_id' : '',
            'peturbing_orf' : None,
            'context' : {
                'ref' : None,
                'alt' : new_context,
            },
            'efficiency':{
                'ref_efficiency' : None,
                'alt_efficiency' : te[new_context].values[0]
            },
            'intervals' :{
                'start' : new_atg_pos,
                'end' : stop_site,
            }
        }
    # UTR Annotator doesn't produce an impact on uAUG lost so we need to get to 
    # find a way to look at them 
    elif consequence == 'uAUG_lost':
        'annotation-id' : '',
        'peturbing_orf' : None,
            # Find the orf that is perturbed
        'context' : {
            'ref' : None,
            'alt' : None,
        },
        'efficiency':{
            'ref_efficiency' : None,
            'alt_efficiency' : None
        },
        'intervals' :{
            'start' : ,
            'end' : ,
        }
    elif consequence == 'uStop_gained':




    
seq, start_site = get_sequence_and_start('ENST00000358273')
mutated = mutate_sequence(115, 'c', 'g', 'SNV', seq)
mutated[112-6:112+5]

SyntaxError: illegal target for annotation (2707551271.py, line 50)